In [1]:
from pathlib import Path
import sys
sys.path.append("c:\\Users\\kpdav\\machine_learning\\projects\\PGA-portfolio-optimizer\\config")
sys.path.append("c:\\Users\\kpdav\\machine_learning\\projects\\PGA-portfolio-optimizer\\src\\data")

import config

import pandas as pd
pd.options.display.max_columns = 999
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
timeseries_feature_path = str(Path(config.TIMESERIES_FRAMEWORK_DIR, "train_dataset_default.csv"))
df = pd.read_csv(timeseries_feature_path, parse_dates=["date"])

feature_store_path = str(Path(config.TIMESERIES_FRAMEWORK_DIR, "ts_feature_store.csv"))
feature_df = pd.read_csv(feature_store_path, parse_dates=["date"])

draftkings_prices_path = str(Path(config.RAW_HISTORICAL_PRICES, "DKSalaries.csv"))
price_df = pd.read_csv(draftkings_prices_path)

In [3]:
feature_df.head()

,player_id,tournament_id,date,round_1_1,round_1_2,round_1_3,round_1_4,round_1_5,round_1_6,round_1_7,round_1_8,round_1_9,round_1_10,round_1_11,round_1_12,round_1_13,round_1_14,round_1_15,round_1_16,round_1_17,round_1_18,round_2_1,round_2_2,round_2_3,round_2_4,round_2_5,round_2_6,round_2_7,round_2_8,round_2_9,round_2_10,round_2_11,round_2_12,round_2_13,round_2_14,round_2_15,round_2_16,round_2_17,round_2_18,round_3_1,round_3_2,round_3_3,round_3_4,round_3_5,round_3_6,round_3_7,round_3_8,round_3_9,round_3_10,round_3_11,round_3_12,round_3_13,round_3_14,round_3_15,round_3_16,round_3_17,round_3_18,round_4_1,round_4_2,round_4_3,round_4_4,round_4_5,round_4_6,round_4_7,round_4_8,round_4_9,round_4_10,round_4_11,round_4_12,round_4_13,round_4_14,round_4_15,round_4_16,round_4_17,round_4_18,round_1_1_pts,round_1_2_pts,round_1_3_pts,round_1_4_pts,round_1_5_pts,round_1_6_pts,round_1_7_pts,round_1_8_pts,round_1_9_pts,round_1_10_pts,round_1_11_pts,round_1_12_pts,round_1_13_pts,round_1_14_pts,round_1_15_pts,round_1_16_pts,round_1_17_pts,round_1_18_pts,round_2_1_pts,round_2_2_pts,round_2_3_pts,round_2_4_pts,round_2_5_pts,round_2_6_pts,round_2_7_pts,round_2_8_pts,round_2_9_pts,round_2_10_pts,round_2_11_pts,round_2_12_pts,round_2_13_pts,round_2_14_pts,round_2_15_pts,round_2_16_pts,round_2_17_pts,round_2_18_pts,round_3_1_pts,round_3_2_pts,round_3_3_pts,round_3_4_pts,round_3_5_pts,round_3_6_pts,round_3_7_pts,round_3_8_pts,round_3_9_pts,round_3_10_pts,round_3_11_pts,round_3_12_pts,round_3_13_pts,round_3_14_pts,round_3_15_pts,round_3_16_pts,round_3_17_pts,round_3_18_pts,round_4_1_pts,round_4_2_pts,round_4_3_pts,round_4_4_pts,round_4_5_pts,round_4_6_pts,round_4_7_pts,round_4_8_pts,round_4_9_pts,round_4_10_pts,round_4_11_pts,round_4_12_pts,round_4_13_pts,round_4_14_pts,round_4_15_pts,round_4_16_pts,round_4_17_pts,round_4_18_pts,make_cut,total,place,fantasy_placing_pts,f_pts_1_1,f_pts_1_2,f_pts_1_3,f_pts_1_4,f_pts_1_5,f_pts_1_6,f_pts_1_7,f_pts_1_8,f_pts_1_9,f_pts_1_10,f_pts_1_11,f_pts_1_12,f_pts_1_13,f_pts_1_14,f_pts_1_15,f_pts_1_16,f_pts_1_17,f_pts_1_18,f_pts_2_1,f_pts_2_2,f_pts_2_3,f_pts_2_4,f_pts_2_5,f_pts_2_6,f_pts_2_7,f_pts_2_8,f_pts_2_9,f_pts_2_10,f_pts_2_11,f_pts_2_12,f_pts_2_13,f_pts_2_14,f_pts_2_15,f_pts_2_16,f_pts_2_17,f_pts_2_18,f_pts_3_1,f_pts_3_2,f_pts_3_3,f_pts_3_4,f_pts_3_5,f_pts_3_6,f_pts_3_7,f_pts_3_8,f_pts_3_9,f_pts_3_10,f_pts_3_11,f_pts_3_12,f_pts_3_13,f_pts_3_14,f_pts_3_15,f_pts_3_16,f_pts_3_17,f_pts_3_18,f_pts_4_1,f_pts_4_2,f_pts_4_3,f_pts_4_4,f_pts_4_5,f_pts_4_6,f_pts_4_7,f_pts_4_8,f_pts_4_9,f_pts_4_10,f_pts_4_11,f_pts_4_12,f_pts_4_13,f_pts_4_14,f_pts_4_15,f_pts_4_16,f_pts_4_17,f_pts_4_18,fantasy_hole_score_pts,complete_r1,complete_r2,complete_r3,complete_r4,bf1,bf2,bf3,bf4,fantasy_bogeyfree_pts,birdie_streak_r1,birdie_streak_r2,birdie_streak_r3,birdie_streak_r4,fantasy_birdie_streak_pts,fantasy_hole_in_one_pts,rd_total_1,rd_total_2,rd_total_3,rd_total_4,under70_1,under70_2,under70_3,under70_4,fantasy_under70_pts,fantasy_total_points,last_win,days_since_win,last_top_3,days_since_top_3,last_top_5,days_since_top_5,last_top_7,days_since_top_7,last_top_9,days_since_top_9,last_top_11,days_since_top_11,last_top_13,days_since_top_13,last_top_15,days_since_top_15,last_top_17,days_since_top_17,last_top_19,days_since_top_19,last_top_21,days_since_top_21,last_top_23,days_since_top_23,last_top_25,days_since_top_25,last_top_27,days_since_top_27,last_top_29,days_since_top_29,last_top_31,days_since_top_31,last_top_33,days_since_top_33,last_top_35,days_since_top_35,last_top_37,days_since_top_37,last_top_39,days_since_top_39,last_top_41,days_since_top_41,last_top_43,days_since_top_43,last_top_45,days_since_top_45,last_top_47,days_since_top_47,last_top_49,days_since_top_49,made_cut,finished_r1,finished_r2,finished_r3,finished_r4,birdie_to_par_ratio,birdie_to_bogey_ratio,birdie_to_double_ratio,eagle_to_par_ratio,eagle_to_bogey_ratio,eagle_to_double_ratio,par_to_birdie_ratio,par_to_bogey_ratio,par_to_double_ratio,eagle_pct,birdie_pct,par_pct,bogey_pct,dou

In [4]:
# df["made_cut"] = np.where(df["make_cut"] == True, 1, 0)
# df["finished_r1"] = np.where(df["complete_r1"] == True, 1, 0)
# df["finished_r2"] = np.where(df["complete_r2"] == True, 1, 0)
# df["finished_r3"] = np.where(df["complete_r3"] == True, 1, 0)
# df["finished_r4"] = np.where(df["complete_r4"] == True, 1, 0)

# def hole_scoring_percentages(p_round, score_type_1, score_type_2=None):
#     """Get hole scoring percentage for a given score type. Or
#     if two score types are given, find the ratio between them
    
#     Args:
#         p_round (series or pd.DataFrame) : player round score(s)
        
#         score_type_1 (int) : shooting score type
        
#         score_type_2 (int) : shooting score type, optional argument
        
#     Returns:
#         percentage for given scoring type(s)"""

    
#     if score_type_2 is not None:
        
#         s_type1_count = np.sum(np.where(p_round.values==score_type_1, 1, 0))
#         s_type2_count = np.sum(np.where(p_round.values==score_type_2, 1, 0))

#         if s_type2_count == 0:
#             return 0
#         else:
#             s_percentage = np.round(s_type1_count / s_type2_count, 5)
#             return s_percentage
        
#     else:
        
#         s_type_count = np.sum(np.where(p_round.values==score_type_1, 1, 0))
#         return s_type_count / p_round.shape[0]

# fantasy_hole_pt_cols = df.loc[:, "f_pts_1_1":"f_pts_4_18"].columns.tolist()


# df["birdie_to_par_ratio"] = df[fantasy_hole_pt_cols].apply(lambda x: hole_scoring_percentages(x, 3, 0.5), axis=1)
# df["birdie_to_bogey_ratio"] = df[fantasy_hole_pt_cols].apply(lambda x: hole_scoring_percentages(x, 3, -0.5), axis=1)
# df["birdie_to_double_ratio"] = df[fantasy_hole_pt_cols].apply(lambda x: hole_scoring_percentages(x, 3, -1), axis=1)

# df["eagle_to_par_ratio"] = df[fantasy_hole_pt_cols].apply(lambda x: hole_scoring_percentages(x, 8, 0.5), axis=1)
# df["eagle_to_bogey_ratio"] = df[fantasy_hole_pt_cols].apply(lambda x: hole_scoring_percentages(x, 8, -0.5), axis=1)
# df["eagle_to_double_ratio"] = df[fantasy_hole_pt_cols].apply(lambda x: hole_scoring_percentages(x, 8, -1), axis=1)

# df["par_to_birdie_ratio"] = df[fantasy_hole_pt_cols].apply(lambda x: hole_scoring_percentages(x, 0.5, 3), axis=1)
# df["par_to_bogey_ratio"] = df[fantasy_hole_pt_cols].apply(lambda x: hole_scoring_percentages(x, 0.5, -0.5), axis=1)
# df["par_to_double_ratio"] = df[fantasy_hole_pt_cols].apply(lambda x: hole_scoring_percentages(x, 0.5, -1), axis=1)

# df["eagle_pct"] = df[fantasy_hole_pt_cols].apply(lambda x: hole_scoring_percentages(x, 8), axis=1)
# df["birdie_pct"] = df[fantasy_hole_pt_cols].apply(lambda x: hole_scoring_percentages(x, 3), axis=1)
# df["par_pct"] = df[fantasy_hole_pt_cols].apply(lambda x: hole_scoring_percentages(x, 0.5), axis=1)
# df["bogey_pct"] = df[fantasy_hole_pt_cols].apply(lambda x: hole_scoring_percentages(x, -0.5), axis=1)
# df["double_pct"] = df[fantasy_hole_pt_cols].apply(lambda x: hole_scoring_percentages(x, -1), axis=1)


# redundant_columns = ["make_cut", "complete_r1", "complete_r2", "complete_r3", "complete_r4"]

# def get_high_correlated_features(df, ignore_cols):
#     """get highest 18 abs value of correlated features
    
#     Args:
#         df

#         ignore_cols
#     Returns:

#     """
#     corr_matrix = df.corr()
#     f_total_pts_correlations = corr_matrix["fantasy_total_points"].abs().sort_values(ascending=False)
#     f_total_pts_correlations.dropna(inplace=True)

#     f_total_pts_correlations = f_total_pts_correlations[~f_total_pts_correlations.index.isin(redundant_columns)]
    
#     return f_total_pts_correlations[:19] 
    

# hc_features = get_high_correlated_features(df, redundant_columns)

# hc_feature_cols = hc_features.index.to_list()
# missing_feat_cols = df[hc_feature_cols].columns[df[hc_feature_cols].isna().any()].tolist()

# # Deal with missing values
# df["total"].fillna(df["total"].max()+20, inplace=True)
# df["place"].fillna(156, inplace=True)
# df["bf2"].fillna(df["bf2"].median()+1, inplace=True)
# df["bf1"].fillna(df["bf1"].median()+1, inplace=True)

In [68]:
out_of_sample_tourns = [401148239]   
in_sample_tourns = df["tournament_id"][df["tournament_id"] != out_of_sample_tourns[0]].unique()

in_sample_df = df[df["tournament_id"].isin(in_sample_tourns)].copy()

in_sample_df["r_avg(2)_tournament_score"] = in_sample_df["fantasy_total_points"].rolling(2, min_periods=2).mean()
in_sample_df = in_sample_df[in_sample_df["r_avg(2)_tournament_score"].notna()]

this_week_competitors = df["player_id"][df.tournament_id == out_of_sample_tourns[0]].unique()

In [144]:
test_df = df[df["tournament_id"].isin(in_sample_tourns)].copy()

In [149]:
np.sort(test_df.player_id.unique())

array([      2,       5,      11, ..., 4592199, 4592200, 4592201],
      dtype=int64)

In [150]:
test_df.pivot(index="player_id", columns="tournament_id", values="fantasy_total_points").fillna(0)

tournament_id,2689,2690,2691,2692,2693,2694,2695,2696,2697,2699,2700,2701,2702,2703,2704,2705,2706,2707,2708,2709,2710,2711,2712,2713,2714,2715,2716,2717,2718,2719,2722,2723,2724,2725,2726,2727,2728,2729,2730,3065,3066,3067,3068,3069,3735,3742,3743,3744,3745,3746,3747,3748,3749,3750,3756,3763,3768,3771,3774,3776,3777,3780,3781,3789,3791,3802,3803,401025221,401025246,401025247,401025249,401025250,401025251,401025252,401025253,401025254,401025255,401025256,401025257,401025258,401025259,401025260,401025261,401025263,401025264,401025265,401025266,401025267,401025268,401025270,401025271,401056252,401056500,401056501,401056502,401056503,401056504,401056505,401056506,401056507,401056510,401056511,401056512,401056513,401056514,401056515,401056516,401056517,401056520,401056521,401056522,401056523,401056525,401056526,401056527,401056528,401056543,401056544,401056545,401056546,401056547,401056548,401056549,401056550,401056551,401056552,401056553,401056554,401056555,401056556,401056557,401056558,401056559,401077167,401148233,401148234,401148235,401148236,401148237,401148238
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.5,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.5,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.5,0.0,0.0,0.0,0.0,0.0,64.0,0.0,0.0,24.0,0.0,0.0,20.0,20.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,22.5,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.5,0.0,0.0,15.5,0.0
11,0.0,0.0,0.0,45.5,18.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,12.0,16.0,0.0,0.0,21.0,11.0,0.0,0.0,60.5,0.0,0.0,0.0,95.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.5,15.5,0.0,31.5,44.5,0.0,0.0,17.5,0.0,0.0,0.0,0.0,0.0,0.0,35.5,0.0,0.0,0.0,0.0,24.0,0.0,68.0,0.0,0.0,0.0,0.0,0.0,0.0,20.5,0.0,0.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,33.5,0.0,0.0,78.5,0.0,0.0,26.5,73.5,0.0,0.0,0.0,0.0,38.5,0.0,0.0,0.0,0.0,0.0,25.5,0.0,0.0,0.0,0.0,63.5,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,74.0,72.5,28.5,0.0,26.0,0.0,19.5,0.0,67.0,72.5,0.0,24.0,52.5,78.5,0.0,18.5,25.0,0.0,0.0,0.0,71.0,0.0,0.0,0.0,0.0,0.0,28.5,95.0,0.0,23.0,0.0,0.0,0.0,63.0,0.0,0.0,93.5,0.0,0.0,0.0,0.0,0.0,17.5,0.0,0.0,0.0,0.0,77.5,68.0,29.5,23.5,81.0,0.0,0.0,63.0,70.5,90.0,53.5,0.0,58.5,29.0,0.0,29.5,0.0,0.0,0.0,0.0,0.0,71.0,91.5,20.0,0.0,28.0,18.0,0.0,63.5,46.5,23.0,0.0,31.0,0.0,79.0,0.0,0.0,92.5,0.0,0.0,0.0,0.0,25.5,49.0,99.5,0.0,0.0,0.0,28.5,0.0,0.0,83.0,0.0,0.0,103.5,0.0,24.5,37.0,69.0,0.0,104.5,0.0,78.5,28.0,0.0,98.0,79.5,0.0,34.5,0.0,2.0,0.0,0.0,0.0,0.0,26.5,0.0,0.0,0.0,56.5,75.5,0.0,26.5,0.0,0.0,0.0,0.0,0.0,0.0,72.5,96.0,23.5,0.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [153]:
pivot_df = test_df.pivot(index="player_id", columns="date", values="fantasy_total_points").fillna(0)


In [156]:
pivot_df.T.corr()

player_id   2         5         11        16        21        28       \
player_id                                                               
2          1.000000  0.026367 -0.075262 -0.069143 -0.027336 -0.041927   
5          0.026367  1.000000  0.487345  0.157167  0.695945  0.761213   
11        -0.075262  0.487345  1.000000  0.153782  0.394041  0.435375   
16        -0.069143  0.157167  0.153782  1.000000  0.074728  0.108898   
21        -0.027336  0.695945  0.394041  0.074728  1.000000  0.393898   
...             ...       ...       ...       ...       ...       ...   
4590403   -0.014958 -0.025857 -0.037550  0.181748 -0.013639 -0.020918   
4591109   -0.014958  0.071808 -0.037550 -0.006471 -0.013639 -0.020918   
4592199   -0.014958 -0.025857 -0.037550 -0.067480 -0.013639 -0.020918   
4592200   -0.014958 -0.025857 -0.037550 -0.067480 -0.013639 -0.020918   
4592201   -0.014958 -0.025857 -0.037550 -0.067480 -0.013639 -0.020918   

player_id   29        34        37        41        47        49       \
player_id                                                               
2         -0.026857 -0.023336 -0.014958 -0.014958 -0.020638 -0.027529   
5          0.028647  0.410823 -0.025857  0.141118 -0.035677  0.512213   
11        -0.067422  0.302358 -0.037550  0.075396 -0.051810  0.272569   
16        -0.074265  0.046809 -0.067480  0.059730 -0.093106  0.009591   
21        -0.024488  0.029804 -0.013639 -0.013639 -0.018818  0.626319   
...             ...       ...       ...       ...       ...       ...   
4590403   -0.013399 -0.011643 -0.007463 -0.007463 -0.010297 -0.013735   
4591109    0.761026 -0.011643 -0.007463 -0.007463 -0.010297 -0.013735   
4592199   -0.013399 -0.011643 -0.007463 -0.007463 -0.010297 -0.013735   
4592200   -0.013399 -0.011643 -0.007463 -0.007463 -0.010297 -0.013735   
4592201   -0.013399 -0.011643 -0.007463 -0.007463 -0.010297 -0.013735   

player_id   53        56        58        65        66        68       \
player_id                                                               
2         -0.014958 -0.021153 -0.014958 -0.058306 -0.014958  0.015918   
5         -0.025857 -0.036567  0.141118  0.378209  0.141118  0.222864   
11        -0.037550 -0.053102  0.075396  0.304623  0.075396  0.274152   
16         0.190835  0.026157  0.059730  0.099783  0.059730  0.235965   
21        -0.013639 -0.019287 -0.013639  0.410309 -0.013639  0.103037   
...             ...       ...       ...       ...       ...       ...   
4590403   -0.007463 -0.010554 -0.007463 -0.029090 -0.007463 -0.070417   
4591109   -0.007463 -0.010554 -0.007463 -0.029090 -0.007463  0.164206   
4592199   -0.007463 -0.010554 -0.007463 -0.029090 -0.007463 -0.070417   
4592200   -0.007463 -0.010554 -0.007463 -0.029090 -0.007463 -0.070417   
4592201   -0.007463 -0.010554 -0.007463 -0.029090 -0.007463 -0.070417   

player_id   72        74        77        78        82        83       \
player_id                                                               
2         -0.106630  0.007423 -0.009599  0.120319  0.288182 -0.025852   
5         -0.012369  0.049137  0.100453 -0.067355  0.420929  0.104476   
11        -0.148669  0.203070  0.196078 -0.061949  0.065100  0.041318   
16        -0.113863  0.140121  0.129489 -0.130163  0.014869  0.000788   
21         0.020191  0.155063  0.105328 -0.109123  0.349359 -0.023572   
...             ...       ...       ...       ...       ...       ...   
4590403   -0.072437 -0.050998 -0.049613 -0.075187 -0.011266 -0.012898   
4591109   -0.072437 -0.050998 -0.049613  0.154578 -0.011266 -0.012898   
4592199   -0.072437 -0.050998  0.248212 -0.075187 -0.011266 -0.012898   
4592200   -0.072437 -0.050998  0.248212 -0.075187 -0.011266 -0.012898   
4592201   -0.072437 -0.050998  0.248212 -0.075187 -0.011266 -0.012898   

player_id   91        97        100       109       110       115      \
player_id                                                               
2         -0.033050  0.250681 -0.014958 -0.023049  0.2

In [102]:
competitors_df = in_sample_df[["player_id", "tournament_id", "fantasy_total_points"]][in_sample_df.player_id.isin(this_week_competitors)]

In [103]:
m_mask = np.isin(this_week_competitors, competitors_df["player_id"].unique())

remove_pids = this_week_competitors[~m_mask]

out_sample_df = df[df["tournament_id"] == out_of_sample_tourns[0]].copy()

remove_pid_idxs = out_sample_df[out_sample_df.player_id.isin(remove_pids)].index

out_sample_df.drop(index=remove_pid_idxs, inplace=True)

In [104]:
competitors_std = competitors_df[["player_id", "fantasy_total_points"]].groupby("player_id").agg(
    std = pd.NamedAgg(column="fantasy_total_points", aggfunc="std"),
)
competitors_std.dropna(inplace=True)

In [105]:
sub_copy_df = competitors_df.copy()

In [98]:
competitors_df.player_id.unique()

array([    318,    1037,    8973,    1185,    5548,    6017,     707,
          4047,    1293,    9037,     780,    6196,     301,    9261,
          9413,    5724,   11119,    5057,    1600,    5502, 4350033,
          6015, 4364873], dtype=int64)

In [109]:
pid_dict = {}
for row in sub_copy_df.itertuples(index=False):
    # print(row, row[0])
    kv_pair = {row[1] : row[2]}
    try:
        pid_dict[row[0]].update(kv_pair)
    except KeyError as e:
        pid_dict[row[0]] = kv_pair

for k,v in pid_dict.items():
    # Find the differences between two sets and sort by t_id
    tourn_set = set(in_sample_tourns)
    result = set(v.keys())
    missing_tourns = tourn_set.difference(result)
    missing_tourns = np.sort(list(missing_tourns))
    
    # add missing tourns with a value of 0 for each player
    m_tourn_dict = dict.fromkeys(missing_tourns, 0)
    pid_dict[k].update(m_tourn_dict)

In [120]:
filled_df = pd.DataFrame.from_dict(pid_dict)
filled_df = filled_df.T.sort_index().T
corr_df = filled_df.corr()
np_cov = np.cov(filled_df, rowvar=False)
np_corr = np.corrcoef(filled_df, rowvar=False)


In [122]:
np.fill_diagonal(corr_df.values, competitors_std)

In [123]:
corr_df

,301,318,707,780,1037,1185,1293,1600,4047,5057,5502,5548,5724,6015,6017,6196,8973,9037,9261,9413,11119,4350033,4364873
301,26.014684,-0.142990,-0.077512,-0.175564,0.044151,0.036915,-0.095557,0.042583,0.098264,-0.084465,0.038775,-0.006256,-0.041282,-0.029862,0.055347,0.144042,0.086605,0.043336,0.114567,0.124009,0.120989,-0.061304,-0.039192
318,-0.142990,33.233747,0.202043,0.157203,0.288487,0.032365,0.107260,-0.013078,0.038049,0.024172,0.154561,0.152506,-0.018158,-0.064873,0.246803,0.002789,-0.145423,0.066117,0.090742,0.079207,-0.179227,-0.076157,-0.059192
707,-0.077512,0.202043,31.234482,-0.040347,0.103391,0.033256,0.099050,-0.104427,0.076709,0.012052,-0.057626,0.415735,0.001501,-0.051708,0.064500,-0.178612,-0.067596,-0.016626,-0.005792,0.214883,-0.113567,-0.004111,0.041799
780,-0.175564,0.157203,-0.040347,29.048792,-0.157467,-0.066804,0.299686,-0.144220,-0.169145,0.132066,-0.084371,0.074911,0.038207,-0.041637,-0.023928,-0.200191,-0.155520,0.207287,-0.036874,-0.087306,-0.196627,-0.002688,0.012292
1037,0.044151,0.288487,0.103391,-0.157467,31.117047,0.174155,-0.095165,0.196319,0.052353,-0.147044,0.252910,0.196598,-0.113210,0.114985,0.240977,0.151851,0.030482,-0.039582,0.315678,0.077798,0.057435,0.108221,0.107671
1185,0.036915,0.032365,0.033256,-0.066804,0.174155,24.818406,-0.193416,0.247402,0.025261,-0.188332,0.103914,0.035055,-0.155661,0.285101,0.263213,0.216771,0.251491,-0.266892,0.192671,-0.051926,0.357606,-0.033707,0.111694
1293,-0.095557,0.107260,0.099050,0.299686,-0.095165,-0.193416,24.566560,-0.203610,-0.054571,0.139987,0.071674,0.126632,0.012494,-0.051488,0.037712,-0.121222,-0.123356,0.344463,0.148183,-0.034080,-0.140707,-0.052181,-0.026463
1600,0.042583,-0.013078,-0.104427,-0.144220,0.196319,0.247402,-0.203610,31.698497,-0.186273,-0.092695,0.359272,-0.105340,-0.045553,0.450541,-0.007309,0.203090,0.296424,-0.119875,0.165574,0.041606,0.409664,-0.049585,0.323730
4047,0.098264,0.038049,0.076709,-0.169145,0.052353,0.025261,-0.054571,-0.186273,26.077756,-0.068140,-0.225635,0.166356,-0.056320,-0.150167,0.115777,-0.111449,-0.014229,-0.123773,-0.186478,-0.059044,-0.153483,-0.031676,-0.100181
5057,-0.084465,0.024172,0.012052,0.132066,-0.147044,-0.188332,0.139987,-0.092695,-0.068140,16.397154,-0.079913,0.036587,0.400371,-0.058154,-0.175113,-0.138798,-0.081030,0.249260,-0.022965,0.098735,-0.101996,-0.018139,-0.057367


In [124]:
competitors_std

,std
player_id,
301,26.014684
318,33.233747
707,31.234482
780,29.048792
1037,31.117047
1185,24.818406
1293,24.566560
1600,31.698497
4047,26.077756


In [8]:
# Look at one tournament and figure out how to perform the task of computing the vector of means and covariance matrix
# Given that I would need to run, train, optimize, etc, the model. I will forgo that and continue onto the covariance matrix step

zozo_championship = df[df.tournament_id==401148239].copy()
zozo_championship.head()

,player_id,tournament_id,date,round_1_1,round_1_2,round_1_3,round_1_4,round_1_5,round_1_6,round_1_7,round_1_8,round_1_9,round_1_10,round_1_11,round_1_12,round_1_13,round_1_14,round_1_15,round_1_16,round_1_17,round_1_18,round_2_1,round_2_2,round_2_3,round_2_4,round_2_5,round_2_6,round_2_7,round_2_8,round_2_9,round_2_10,round_2_11,round_2_12,round_2_13,round_2_14,round_2_15,round_2_16,round_2_17,round_2_18,round_3_1,round_3_2,round_3_3,round_3_4,round_3_5,round_3_6,round_3_7,round_3_8,round_3_9,round_3_10,round_3_11,round_3_12,round_3_13,round_3_14,round_3_15,round_3_16,round_3_17,round_3_18,round_4_1,round_4_2,round_4_3,round_4_4,round_4_5,round_4_6,round_4_7,round_4_8,round_4_9,round_4_10,round_4_11,round_4_12,round_4_13,round_4_14,round_4_15,round_4_16,round_4_17,round_4_18,round_1_1_pts,round_1_2_pts,round_1_3_pts,round_1_4_pts,round_1_5_pts,round_1_6_pts,round_1_7_pts,round_1_8_pts,round_1_9_pts,round_1_10_pts,round_1_11_pts,round_1_12_pts,round_1_13_pts,round_1_14_pts,round_1_15_pts,round_1_16_pts,round_1_17_pts,round_1_18_pts,round_2_1_pts,round_2_2_pts,round_2_3_pts,round_2_4_pts,round_2_5_pts,round_2_6_pts,round_2_7_pts,round_2_8_pts,round_2_9_pts,round_2_10_pts,round_2_11_pts,round_2_12_pts,round_2_13_pts,round_2_14_pts,round_2_15_pts,round_2_16_pts,round_2_17_pts,round_2_18_pts,round_3_1_pts,round_3_2_pts,round_3_3_pts,round_3_4_pts,round_3_5_pts,round_3_6_pts,round_3_7_pts,round_3_8_pts,round_3_9_pts,round_3_10_pts,round_3_11_pts,round_3_12_pts,round_3_13_pts,round_3_14_pts,round_3_15_pts,round_3_16_pts,round_3_17_pts,round_3_18_pts,round_4_1_pts,round_4_2_pts,round_4_3_pts,round_4_4_pts,round_4_5_pts,round_4_6_pts,round_4_7_pts,round_4_8_pts,round_4_9_pts,round_4_10_pts,round_4_11_pts,round_4_12_pts,round_4_13_pts,round_4_14_pts,round_4_15_pts,round_4_16_pts,round_4_17_pts,round_4_18_pts,make_cut,total,place,fantasy_placing_pts,f_pts_1_1,f_pts_1_2,f_pts_1_3,f_pts_1_4,f_pts_1_5,f_pts_1_6,f_pts_1_7,f_pts_1_8,f_pts_1_9,f_pts_1_10,f_pts_1_11,f_pts_1_12,f_pts_1_13,f_pts_1_14,f_pts_1_15,f_pts_1_16,f_pts_1_17,f_pts_1_18,f_pts_2_1,f_pts_2_2,f_pts_2_3,f_pts_2_4,f_pts_2_5,f_pts_2_6,f_pts_2_7,f_pts_2_8,f_pts_2_9,f_pts_2_10,f_pts_2_11,f_pts_2_12,f_pts_2_13,f_pts_2_14,f_pts_2_15,f_pts_2_16,f_pts_2_17,f_pts_2_18,f_pts_3_1,f_pts_3_2,f_pts_3_3,f_pts_3_4,f_pts_3_5,f_pts_3_6,f_pts_3_7,f_pts_3_8,f_pts_3_9,f_pts_3_10,f_pts_3_11,f_pts_3_12,f_pts_3_13,f_pts_3_14,f_pts_3_15,f_pts_3_16,f_pts_3_17,f_pts_3_18,f_pts_4_1,f_pts_4_2,f_pts_4_3,f_pts_4_4,f_pts_4_5,f_pts_4_6,f_pts_4_7,f_pts_4_8,f_pts_4_9,f_pts_4_10,f_pts_4_11,f_pts_4_12,f_pts_4_13,f_pts_4_14,f_pts_4_15,f_pts_4_16,f_pts_4_17,f_pts_4_18,fantasy_hole_score_pts,complete_r1,complete_r2,complete_r3,complete_r4,bf1,bf2,bf3,bf4,fantasy_bogeyfree_pts,birdie_streak_r1,birdie_streak_r2,birdie_streak_r3,birdie_streak_r4,fantasy_birdie_streak_pts,fantasy_hole_in_one_pts,rd_total_1,rd_total_2,rd_total_3,rd_total_4,under70_1,under70_2,under70_3,under70_4,fantasy_under70_pts,fantasy_total_points,last_win,days_since_win,last_top_3,days_since_top_3,last_top_5,days_since_top_5,last_top_7,days_since_top_7,last_top_9,days_since_top_9,last_top_11,days_since_top_11,last_top_13,days_since_top_13,last_top_15,days_since_top_15,last_top_17,days_since_top_17,last_top_19,days_since_top_19,last_top_21,days_since_top_21,last_top_23,days_since_top_23,last_top_25,days_since_top_25,last_top_27,days_since_top_27,last_top_29,days_since_top_29,last_top_31,days_since_top_31,last_top_33,days_since_top_33,last_top_35,days_since_top_35,last_top_37,days_since_top_37,last_top_39,days_since_top_39,last_top_41,days_since_top_41,last_top_43,days_since_top_43,last_top_45,days_since_top_45,last_top_47,days_since_top_47,last_top_49,days_since_top_49,made_cut,finished_r1,finished_r2,finished_r3,finished_r4,birdie_to_par_ratio,birdie_to_bogey_ratio,birdie_to_double_ratio,eagle_to_par_ratio,eagle_to_bogey_ratio,eagle_to_double_ratio,par_to_birdie_ratio,par_to_bogey_ratio,par_to_double_ratio,eagle_pct,birdie_pct,par_pct,bogey_pct,dou

In [9]:
players = zozo_championship.player_id.values
previous_5_tournaments = df.tournament_id.unique()[-6:-1]

In [17]:
historical_5_df = df[(df.tournament_id.isin(previous_5_tournaments))].copy()

In [23]:
sub_historical_5 = historical_5_df[historical_5_df.player_id.isin(players)]
sub_historical_5.player_id.unique().shape

(17,)

In [15]:
players.shape

(25,)

In [ ]:
# from above we see that not everyone played in the last 5 tournaments so it should be individual to each player 
# competiting or some way to handle players who don't play as freqeuently

In [7]:
def rolling_mean_score(scoring, days):
    """Compute rolling mean fantasy score over given number of days
    
    Args:
        scoring (pd.DataFrame) : DataFrame consisting of player_id and
                                fantasy_total_points
        
        days (int) : rolling period to be used
    
    Returns:
        rolling mean score over given days
    """
    return scoring.groupby("player_id")[["fantasy_total_points"]].rolling(days, min_periods=days).mean()

In [8]:
# How to create rolling mean feature 

# trailing_3_dk_score = df.groupby("player_id")[["fantasy_total_points"]].rolling(3, min_periods=3).mean()
# trailing_3_dk_score = trailing_3_dk_score[["fantasy_total_points"]].reset_index()
# trailing_3_dk_score.set_index("level_1", inplace=True)
# df.loc[trailing_3_dk_score.index.values, "mean_rolling(3)_dk_ftp"] = trailing_3_dk_score["fantasy_total_points"]

In [27]:
# Goals 

# Get list of players competiting in tournament

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,G,Jon Rahm (18319572),Jon Rahm,18319572,G,11300,The Open Championship,Golf,83.82
1,G,Rory McIlroy (18319573),Rory McIlroy,18319573,G,10900,The Open Championship,Golf,69.14
2,G,Brooks Koepka (18319574),Brooks Koepka,18319574,G,10700,The Open Championship,Golf,67.40
3,G,Dustin Johnson (18319575),Dustin Johnson,18319575,G,10400,The Open Championship,Golf,77.47
4,G,Xander Schauffele (18319576),Xander Schauffele,18319576,G,10000,The Open Championship,Golf,82.25
...,...,...,...,...,...,...,...,...,...
170,G,Daniel Croft (18319728),Daniel Croft,18319728,G,6000,The Open Championship,Golf,0.00
171,G,John Daly (18319729),John Daly,18319729,G,6000,The Open Championship,Golf,41.50
172,G,Sam Forgan (18319730),Sam Forgan,18319730,G,6000,The Open Championship,Golf,0.00
173,G,Todd Hamilton (18319731),Todd Hamilton,18319731,G,6000,The Open Championship,Golf,0.00
